In [2]:
!pip install scikit-optimize

    100% |████████████████████████████████| 81kB 3.1MB/s 


In [3]:
!pip install comet_ml
!pip install urllib==1.34


    100% |████████████████████████████████| 92kB 3.5MB/s 
    100% |████████████████████████████████| 204kB 25.6MB/s 
    100% |████████████████████████████████| 389kB 26.0MB/s 
    100% |████████████████████████████████| 122kB 27.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj
requests 2.18.4 has requirement urllib3<1.23,>=1.21.1, but you'll have urllib3 1.24.1 which is incompatible.
fastai 1.0.49 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.22
    Uninstalling urllib3-1.22:
      Successfully uninstalled urllib3-1.22


  Could not find a version that satisfies the requirement urllib==1.34 (from versions: )
No matching distribution found for urllib==1.34


In [1]:
!pip install skorch

    100% |████████████████████████████████| 102kB 2.7MB/s 


In [0]:
from comet_ml import Experiment

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
print(torch.__version__)

1.0.1.post2


## Fashion MINST dataset


In [0]:
from torchvision.datasets import FashionMNIST

In [6]:
# Getting the train and test parts of the dataset
data_train = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=True)

data_test = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=False)

# In fact, it's already stored as torch tensor, but we'll need
# to work with the numpy representation, so let's do the convertion:
X_train = data_train.train_data.numpy()
y_train = data_train.train_labels.numpy()

X_test = data_test.test_data.numpy()
y_test = data_test.test_labels.numpy()


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


The datasets consists of images belonging to one out of 10 classes:

| Label | Description | | Label | Description |
| ---       
| 0        | T-shirt/top   || 5        | Sandal         |
| 1        | Trouser        || 6        | Shirt             |
| 2        | Pullover       || 7        | Sneaker       |
| 3        | Dress           || 8        | Bag              |
| 4        | Coat             || 9        | Ankle boot  |






### Input preprocessing

So far our data is held as numpy arrays of unsigned byte type, i.e. it lies within a range from 0 to 255. Also, the shape of our input is 3-dimensional (num_images, height, width), while our `model` takes 2-dimensional "arrays of 1-dimensional images" (num_images, height * width).

We have to convert that to `torch` tensors and reshape the input. Also, it's a good idea to normalize your image data to lie within a $[0, 1]$ interval. Let's write a function that does all these things:

In [0]:
# Write a function to convert X and y to torch tensors while
# rescaling X to fit into [0, 1] interval and reshaping it properly

# Hint: make sure your input tensor dtype is same as the
# parameters of the model (should be torch.float)

def preprocess_data(X, y):
  X_preprocessed = torch.reshape(torch.from_numpy(X/ 255.).type(torch.float32), (-1, 784))
  y_preprocessed = torch.from_numpy(y)
  return X_preprocessed, y_preprocessed

In [0]:
import torch.nn.functional as F

In [0]:
torch.manual_seed(0);
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
X_train, y_train = preprocess_data(X_train, y_train)


IndexError: ignored

In [0]:
mnist_dim = X_train.shape[1]
hidden_dim = int(mnist_dim/8)
output_dim = len(np.unique(y_train))

In [20]:

mnist_dim, hidden_dim, output_dim

(784, 98, 10)

In [0]:
from torch import nn

class ClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=mnist_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

In [0]:
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device=device,
)

In [24]:
net.fit(X_train, y_train);

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.8887       0.8020        0.5681  1.3593
      2        0.5982       0.8290        0.4875  1.2904
      3        0.5352       0.8388        0.4517  1.2468
      4        0.4983       0.8489        0.4240  1.2428
      5        0.4802       0.8487        0.4160  1.2545
      6        0.4621       0.8591        0.3960  1.2390
      7        0.4518       0.8601        0.3869  1.2682
      8        0.4414       0.8643        0.3811  1.2680
      9        0.4336       0.8639        0.3765  1.2762
     10        0.4241       0.8660        0.3703  1.2779
     11        0.4178       0.8662        0.3654  1.2374
     12        0.4145       0.8691        0.3620  1.2410
     13        0.4084       0.8691        0.3614  1.2410
     14        0.4035       0.8694        0.3589  1.2362
     15        0.3989       0.8698        0.3569  1.2516
     16        0.3950       0.8

In [0]:
X_test, y_test = preprocess_data(X_test, y_test)
predicted = net.predict(X_test)

In [31]:
np.mean(y_test.numpy() == predicted)

0.8618

### Exercise: adding more convolutional magic

```
model = torch.nn.Sequential(
            Reshape(-1, 1, 28, 28),
            torch.nn.Conv2d(in_channels=1,
                            out_channels=8,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(in_channels=8,
                            out_channels=16,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(in_channels=16,
                            out_channels=32,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2, padding=1),
            Reshape(-1, 512),
            torch.nn.Linear(512, 10)
        ).to(device)
```
